<a href="https://colab.research.google.com/github/pallavi-allada/experiments-with-data/blob/master/Image_Classification_FineTuning_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras import models
from keras import layers
from keras.layers import Input,Flatten,Dense,Dropout
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16

Using TensorFlow backend.


Mount Google drive for train,valid and test images

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Download the base model without the head, we will add the head/classifier on top and then fine tune.

In [0]:
TRAIN_DIR = '/content/drive/My Drive/Colab Notebooks/ObjectDetection/images/classification files/train'
VALID_DIR = '/content/drive/My Drive/Colab Notebooks/ObjectDetection/images/classification files/valid'
TEST_DIR = '/content/drive/My Drive/Colab Notebooks/ObjectDetection/images/classification files/test'

img_width = img_height =150
base_model = VGG16(weights='imagenet',include_top=False, input_shape=(img_width,img_height,3))



In [0]:
import numpy as np

train_gen = ImageDataGenerator(rescale=1./255)
val_gen = ImageDataGenerator()
test_gen = ImageDataGenerator()
batchsize=4
trainsize=240
valsize=48

mean = np.array([123.68, 116.779, 103.939], dtype="float32")
train_gen.mean = mean
val_gen.mean = mean
test_gen.mean=mean

Read the data from the train,valid and test folders using ImageDataGenerators


In [23]:
train_data = train_gen.flow_from_directory(directory=TRAIN_DIR,
                                   batch_size=batchsize,
                                   target_size=(img_height,img_width),
                                   shuffle=True,
                                   class_mode='binary')

val_data = val_gen.flow_from_directory(directory=VALID_DIR,
                                 batch_size=batchsize,
                                 class_mode='binary',
                                 target_size=(img_height,img_width),
                                 shuffle=False)

test_data = test_gen.flow_from_directory(directory=TEST_DIR,
                                        batch_size=batchsize,
                                        target_size=(img_height,img_width),
                                        class_mode='binary',
                                        shuffle=False)

Found 240 images belonging to 2 classes.
Found 48 images belonging to 2 classes.
Found 4 images belonging to 1 classes.


Notice the base_model does not have the FC layers as we did not include the top.

In [24]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

Add FC layers to the base model, setting the trainable param of non-FC layers to False



In [33]:
new_model = models.Sequential()
new_model.add(base_model)
new_model.add(Flatten())
new_model.add(layers.Dense(512,activation='relu'))
new_model.add(layers.Dropout(0.5))
new_model.add(layers.Dense(256,activation='relu'))
new_model.add(layers.Dropout(0.5))
new_model.add(layers.Dense(1,activation='sigmoid'))
for layer in new_model.layers[:-6]:
  layer.trainable=False
for layer in new_model.layers:
  print(layer, layer.trainable)

new_model.summary()

<keras.engine.training.Model object at 0x7f83c4e961d0> False
<keras.layers.core.Flatten object at 0x7f83c587bcc0> True
<keras.layers.core.Dense object at 0x7f83c58545c0> True
<keras.layers.core.Dropout object at 0x7f83c5854f28> True
<keras.layers.core.Dense object at 0x7f83c57b2be0> True
<keras.layers.core.Dropout object at 0x7f83c57b2630> True
<keras.layers.core.Dense object at 0x7f83c57e0908> True
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_11 (Flatten)         (None, 8192)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               4194816   
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)      

Compile the model and train for 50 epochs

In [0]:
new_model.compile(loss='binary_crossentropy',optimizer=SGD(lr=1e-03),metrics=['accuracy'])

In [36]:
history = new_model.fit_generator(train_data,
                       steps_per_epoch=trainsize//batchsize,
                       validation_data=val_data,
                       validation_steps=valsize//batchsize,
                       epochs=50)


Epoch 1/50


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


60/60 [==============================] - 6s 100ms/step - loss: 0.7497 - acc: 0.5375 - val_loss: 3.9523 - val_acc: 0.6042
Epoch 2/50
60/60 [==============================] - 5s 80ms/step - loss: 0.6567 - acc: 0.6000 - val_loss: 2.5172 - val_acc: 0.6875
Epoch 3/50
60/60 [==============================] - 5s 83ms/step - loss: 0.5500 - acc: 0.7000 - val_loss: 2.0055 - val_acc: 0.7292
Epoch 4/50
60/60 [==============================] - 5s 83ms/step - loss: 0.5253 - acc: 0.7292 - val_loss: 1.6789 - val_acc: 0.8542
Epoch 5/50
60/60 [==============================] - 5s 81ms/step - loss: 0.5377 - acc: 0.7083 - val_loss: 1.6763 - val_acc: 0.8542
Epoch 6/50
60/60 [==============================] - 5s 81ms/step - loss: 0.4789 - acc: 0.7458 - val_loss: 1.5696 - val_acc: 0.8542
Epoch 7/50
60/60 [==============================] - 5s 82ms/step - loss: 0.4673 - acc: 0.7833 - val_loss: 1.6005 - val_acc: 0.8542
Epoch 8/50
60/60 [==============================] - 5s 82ms/step - loss: 0.4195 - acc: 0.8250

set base_model trainable to True and fine tune as well for 10 epochs.

In [42]:
for layer in base_model.layers[:-4]:
  layer.trainable=True
  
new_model = models.Sequential()
new_model.add(base_model)
new_model.add(Flatten())
new_model.add(layers.Dense(512,activation='relu'))
new_model.add(layers.Dropout(0.5))
new_model.add(layers.Dense(256,activation='relu'))
new_model.add(layers.Dropout(0.5))
new_model.add(layers.Dense(1,activation='sigmoid'))

for layer in new_model.layers:
  layer.trainable=True
  print(layer, layer.trainable)

for layer in new_model.layers[-7].layers:
  print(layer, layer.trainable)
  


<keras.engine.training.Model object at 0x7f83c4e961d0> True
<keras.layers.core.Flatten object at 0x7f83c44e8ba8> True
<keras.layers.core.Dense object at 0x7f83c44e8c18> True
<keras.layers.core.Dropout object at 0x7f83c44e8b70> True
<keras.layers.core.Dense object at 0x7f83c4462320> True
<keras.layers.core.Dropout object at 0x7f83c44625c0> True
<keras.layers.core.Dense object at 0x7f83c43fda90> True
<keras.engine.input_layer.InputLayer object at 0x7f83c5012320> True
<keras.layers.convolutional.Conv2D object at 0x7f83c502a048> True
<keras.layers.convolutional.Conv2D object at 0x7f83c502a1d0> True
<keras.layers.pooling.MaxPooling2D object at 0x7f83c502aac8> True
<keras.layers.convolutional.Conv2D object at 0x7f83c502af28> True
<keras.layers.convolutional.Conv2D object at 0x7f83c4fd5e48> True
<keras.layers.pooling.MaxPooling2D object at 0x7f83c4fe76a0> True
<keras.layers.convolutional.Conv2D object at 0x7f83c4fe77f0> True
<keras.layers.convolutional.Conv2D object at 0x7f83c4f99c50> True
<k

In [44]:
new_model.compile(loss='binary_crossentropy',optimizer=SGD(lr=1e-03),metrics=['accuracy'])
history = new_model.fit_generator(train_data,
                       steps_per_epoch=trainsize//batchsize,
                       validation_data=val_data,
                       validation_steps=valsize//batchsize,
                       epochs=50)


Epoch 1/50


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


60/60 [==============================] - 11s 191ms/step - loss: 0.7560 - acc: 0.5333 - val_loss: 5.2828 - val_acc: 0.5000
Epoch 2/50
60/60 [==============================] - 8s 138ms/step - loss: 0.6005 - acc: 0.7292 - val_loss: 4.6911 - val_acc: 0.5833
Epoch 3/50
60/60 [==============================] - 8s 139ms/step - loss: 0.4677 - acc: 0.7917 - val_loss: 3.4657 - val_acc: 0.6875
Epoch 4/50
60/60 [==============================] - 8s 139ms/step - loss: 0.3703 - acc: 0.8500 - val_loss: 2.4393 - val_acc: 0.8333
Epoch 5/50
60/60 [==============================] - 8s 139ms/step - loss: 0.3152 - acc: 0.8667 - val_loss: 2.2551 - val_acc: 0.8333
Epoch 6/50
60/60 [==============================] - 8s 139ms/step - loss: 0.2025 - acc: 0.9333 - val_loss: 4.1052 - val_acc: 0.7292
Epoch 7/50
60/60 [==============================] - 8s 138ms/step - loss: 0.2630 - acc: 0.9125 - val_loss: 2.5310 - val_acc: 0.8125
Epoch 8/50
60/60 [==============================] - 8s 139ms/step - loss: 0.1810 - acc

In [0]:
# test_data.reset()
# predIdxs = new_model.predict_generator(test_data)
# # predIdxs = np.argmax(predIdxs, axis=1)
# print(predIdxs)
# # print(classification_report(test_data.classes, predIdxs,
# # 	target_names=test_data.class_indices.keys()))
                        